In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors

In [3]:
df=pd.read_csv('Dataset .csv')
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [4]:
df_info = pd.DataFrame({'Column Name': df.columns, 'Data Type': df.dtypes.values})
print(df_info)

             Column Name Data Type
0          Restaurant ID     int64
1        Restaurant Name    object
2           Country Code     int64
3                   City    object
4                Address    object
5               Locality    object
6       Locality Verbose    object
7              Longitude   float64
8               Latitude   float64
9               Cuisines    object
10  Average Cost for two     int64
11              Currency    object
12     Has Table booking    object
13   Has Online delivery    object
14     Is delivering now    object
15  Switch to order menu    object
16           Price range     int64
17      Aggregate rating   float64
18          Rating color    object
19           Rating text    object
20                 Votes     int64


In [5]:
features = ['Average Cost for two', 'Price range', 'Aggregate rating', 'Votes']

In [6]:
df[features] = df[features].fillna(df[features].mean())

In [7]:
scaler = StandardScaler()
feature_matrix = scaler.fit_transform(df[features])

In [8]:
knn_model = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn_model.fit(feature_matrix)

NearestNeighbors(metric='euclidean', n_neighbors=6)

In [9]:
def get_closest_match(restaurant_name, df):
    matches = df[df['Restaurant Name'].str.contains(restaurant_name, case=False, na=False)]['Restaurant Name'].unique()
    return matches[0] if len(matches) > 0 else None

def get_recommendations(restaurant_name, df, model, feature_matrix):
    df['Restaurant Name'] = df['Restaurant Name'].str.strip().str.lower()
    restaurant_name = restaurant_name.strip().lower()
    
    closest_match = get_closest_match(restaurant_name, df)
    
    if closest_match is None:
        return f"⚠️ No restaurant found with the name '{restaurant_name}'!"

    print(f"🔍 Using closest match: {closest_match}")

    idx = df[df['Restaurant Name'] == closest_match].index[0]
    distances, indices = model.kneighbors([feature_matrix[idx]])
    
    recommended_restaurants = df.iloc[indices[0][1:]]
    return recommended_restaurants[['Restaurant Name', 'Aggregate rating', 'Cuisines']]

In [10]:
restaurant_name = "town table"  
recommendations = get_recommendations(restaurant_name, df, knn_model, feature_matrix)
print(recommendations)

🔍 Using closest match: town table restaurant
              Restaurant Name  Aggregate rating  \
9475           old town italy               3.9   
9468                blos cafe               3.9   
861   black pepper restaurant               4.0   
2554           barbeque ville               3.8   
9432                  eatopia               3.9   

                                               Cuisines  
9475       Cafe, Italian, Pizza, European, Bakery, Deli  
9468  Cafe, Burger, Tapas, South African, European, ...  
861                               North Indian, Chinese  
2554                                       North Indian  
9432      European, Arabian, Japanese, Bakery, Desserts  


In [11]:
df[df['Restaurant Name'].str.contains("table", case=False, na=False)]

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
848,3500018,town table restaurant,1,Dehradun,"101, Rajpur Road, Hathibarkala Salwala, Dehradun",Hathibarkala Salwala,"Hathibarkala Salwala, Dehradun",78.060221,30.340722,"North Indian, Chinese, Continental",...,Indian Rupees(Rs.),No,No,No,No,4,3.9,Yellow,Good,173
1729,2071,captain's table,1,Gurgaon,"SCF 96, Main Market, Sector 14, Gurgaon",Sector 14,"Sector 14, Gurgaon",77.047461,28.474027,"Bakery, Desserts",...,Indian Rupees(Rs.),No,No,No,No,1,3.1,Orange,Average,41
9023,312558,the dessert table company,1,Noida,"Flat 203, Tower 6, Parsvnath Prestige 2, Shris...",Sector 93,"Sector 93, Noida",77.386317,28.518998,"Bakery, Desserts",...,Indian Rupees(Rs.),No,No,No,No,2,3.1,Orange,Average,9


In [12]:
print(df['Restaurant Name'].unique)

<bound method Series.unique of 0               le petit souffle
1               izakaya kikufuji
2         heat - edsa shangri-la
3                           ooma
4                    sambo kojin
                  ...           
9546                 naml۱ gurme
9547                ceviz a��ac۱
9548                       huqqa
9549                 a���k kahve
9550    walter's coffee roastery
Name: Restaurant Name, Length: 9551, dtype: object>


In [13]:
import pickle

# Save the trained model (randomreg) as 'randomreg.pkl'
with open("knn_model.pkl", "wb") as file:
    pickle.dump(knn_model, file)

print("Model saved successfully as 'knn_model.pkl'!")

Model saved successfully as 'knn_model.pkl'!
